# **1. 환경 구축**

In [ ]:
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab_light_220429.sh 

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F 
from torch.autograd import Variable 
import pickle 

import MeCab
tagger = MeCab.Tagger()

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Train Data setting

In [ ]:
df=pd.read_csv('/content/gdrive/MyDrive/빅프/last_test_221222.csv',encoding='utf-8', on_bad_lines='skip')


In [ ]:
df = df.drop(['1','2','3'], axis=1)

In [ ]:
df

,text,mbti
0,니모 속박 테니,EN
1,대군 숫돌 테멘,EN
2,데브 승천 텍산,EN
3,리갈 야옹 텐서,EN
4,마차 엔조 토끼,EN
...,...,...
1631,미겔 이케 훼방,IS
1632,미나 인도 흉상,IS
1633,미디 인물 흡충,IS
1634,미소 인주 힌즈,IS


In [ ]:
df.mbti.value_counts()

ES    536
IN    459
EN    352
IS    289
Name: mbti, dtype: int64

In [ ]:
subset_category=["EN", "ES", "IN", "IS"]

df=df[df.mbti.isin(subset_category)]
print (df.head())
print()
print (df.tail())
print()


       text mbti
0  니모 속박 테니   EN
1  대군 숫돌 테멘   EN
2  데브 승천 텍산   EN
3  리갈 야옹 텐서   EN
4  마차 엔조 토끼   EN

          text mbti
1631  미겔 이케 훼방   IS
1632  미나 인도 흉상   IS
1633  미디 인물 흡충   IS
1634  미소 인주 힌즈   IS
1635  미역 인치 힌트   IS



In [ ]:
def mecabsplit(mecab_tagger,inputs, pos):
    r=[]
    inputs = mecab_tagger.parse(inputs)
    t = inputs.split('\n')[:-2]
    for i in t:
        field = i.split('\t')
        if field[1].split(',')[-1].strip() is not '*':
            r.extend( [ (x.split('/')[0],x.split('/')[1]) for x in field[1].split(',')[-1].split('+') ] )
        else:
            r.append( (field[0],field[1].split(',')[0]) )
    if pos:
        return r
    else:
        return [ x[0] for x in r ]
    return r

<>:7: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:7: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<ipython-input-9-1e63fc3c6b9f>:7: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if field[1].split(',')[-1].strip() is not '*':


In [ ]:
content_vocab = {'unk':0}
intent_vocab={}
intent_list=[]

data_intent=''
intent_idx=0
vocab_idx=1

for line in df.mbti.unique():
    intent=line
    if intent not in intent_vocab:
        intent_vocab[intent]=intent_idx
        intent_list.append(intent)
        intent_idx +=1

for line in list(df.text):
    line = mecabsplit(tagger,line,False)
    for it in line:
        if it not in content_vocab:
            content_vocab[it] = vocab_idx # it : 단어
            vocab_idx +=1

In [ ]:
print("vocab size:", len(content_vocab))
print(content_vocab)  # 단어 사전
print(intent_vocab)   # intent 사전
print("intent size:", len(intent_list))
print(intent_list)    # intent_list : FAQ 카테고리 리스트

vocab size: 4747
{'unk': 0, '니모': 1, '속박': 2, '터': 3, '이': 4, '니': 5, '대군': 6, '숫': 7, '돌': 8, '멘': 9, '데브': 10, '승천': 11, '텍': 12, '산': 13, '리갈': 14, '야옹': 15, '서': 16, '마차': 17, '엔조': 18, '토끼': 19, '무수': 20, '오턴': 21, '토니': 22, '다발': 23, '십': 24, '장': 25, '출산': 26, '데프': 27, '아사': 28, '케빈': 29, '도일': 30, '아아': 31, '코모': 32, '두쿠': 33, '아연': 34, '키타': 35, '로': 36, '키': 37, '아자': 38, '트랜': 39, '매듭': 40, '아첨': 41, '표시': 42, '머핀': 43, '탄력': 44, '플': 45, '롭': 46, '뭉킨': 47, '탈선': 48, '현금': 49, '라사': 50, '소피': 51, '초조': 52, '라일': 53, '손': 54, '조': 55, '축소': 56, '랜덤': 57, '수중': 58, '침수': 59, '랜': 60, '도': 61, '시시': 62, '코디': 63, '램지': 64, '에릭': 65, '편달': 66, '랩터': 67, '영혼': 68, '하렘': 69, '딜': 70, '우화': 71, '환경': 72, '십장': 73, '위': 74, '브': 75, '히': 76, '디': 77, '가라': 78, '발기': 79, '유니': 80, '가방': 81, '발달': 82, '유래': 83, '가설': 84, '발리': 85, '유령': 86, '가슴': 87, '발': 88, '릭': 89, '유료': 90, '가정': 91, '발명': 92, '유리': 93, '동면': 94, '목': 95, '유사': 96, '가짜': 97, '발산': 98, '유성': 99, '각도': 100, '방광': 1

## Modeling

In [ ]:
class CNN_Text(nn.Module):
    
    def __init__(self, embed_num, class_num, ):
        super(CNN_Text, self).__init__()
        
        V = embed_num
        D = 200                 # args.embed_dim 단어임베딩 차원
        C = class_num             # 분류할 카테고리 갯수
        Ci = 1                    # 입력 채널의 수
        Co = 50 #args.kernel_num  # 출력 채널의 수
        Ks = [1,1,1]              # 커널(필터) 사이즈 정의


        self.embed = nn.Embedding(V, D)      

        self.convs1 = nn.ModuleList([nn.Conv2d(Ci, Co, (K, D)) for K in Ks])

        self.relu = nn.ReLU(inplace=True)
        
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.dropout = nn.Dropout(0.2)        # dropout 정의
    
        self.fc1 = nn.Linear(len(Ks)*Co, C)   # (60, 5)


    def forward(self, x):
        

        x = self.embed(x)  

             
        x = x.unsqueeze(1)  

        

        x = [F.relu(conv(x)).squeeze(3) for conv in self.convs1]

        

        x = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in x]  # [(N, Co), ...]*len(Ks)

        
        x = torch.cat(x, 1)

        x = self.dropout(x)  


        logit = self.fc1(x)
        
        return logit


In [ ]:
cnn = CNN_Text(vocab_idx,intent_idx)
print(vocab_idx, intent_idx) 
optimizer = torch.optim.Adam(cnn.parameters())
cnn.train()

4747 4


CNN_Text(
  (embed): Embedding(4747, 200)
  (convs1): ModuleList(
    (0): Conv2d(1, 50, kernel_size=(1, 200), stride=(1, 1))
    (1): Conv2d(1, 50, kernel_size=(1, 200), stride=(1, 1))
    (2): Conv2d(1, 50, kernel_size=(1, 200), stride=(1, 1))
  )
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc1): Linear(in_features=150, out_features=4, bias=True)
)

## Training

In [ ]:
epoch = 30
for e in range(epoch):
    totalloss = 0
    
    for line in df.values:
        target=Variable(torch.LongTensor([intent_vocab[line[1]]]))

        optimizer.zero_grad()

        cont = []
        line = mecabsplit(tagger,line[0],False)
        
        for it in line:
            cont.append(content_vocab[it])


        cont = Variable(torch.LongTensor(cont)).view(1,-1)
        pred = cnn(cont)
        
        loss = F.cross_entropy(pred,target)
        #--- end

        totalloss += float(loss.data)
        loss.backward()  
  
        optimizer.step() 
        #--- end
    print(e,'epoch')    
    print('loss : {:.3f}'.format(totalloss))


0 epoch
loss : 2170.964
1 epoch
loss : 2010.899


In [ ]:
torch.save(cnn,'/content/gdrive/My Drive/빅프/cnn_model_v8.pt')

## Test Data setting

<테스트> mbti별 단어를 조합하여 테스트 데이터 생성

In [ ]:
test = pd.read_csv('/content/gdrive/MyDrive/빅프/last_test_test.csv')

In [ ]:
test = test.drop(['1','2','3'], axis=1)

In [ ]:
test

,text,mbti
0,목록 화살 의자,IS
1,목수 화신 이랴,IS
2,목에 화씨 이론,IS
3,목자 환희 이름,IS
4,낮게 순종 초반,IN
5,낳다 순한 초안,IN
6,내선 순회 초월,IN
7,내성 술부 초점,IN
8,마빈 연합 폭죽,ES
9,마샬 열소 폭탄,ES


In [ ]:
#모델불러오기
device = "cuda" if torch.cuda.is_available() else "cpu"
model = torch.load('/content/gdrive/MyDrive/빅프/cnn_model_v7.pt', map_location=device)
model.eval()

CNN_Text(
  (embed): Embedding(4748, 200)
  (convs1): ModuleList(
    (0): Conv2d(1, 50, kernel_size=(1, 200), stride=(1, 1))
    (1): Conv2d(1, 50, kernel_size=(1, 200), stride=(1, 1))
    (2): Conv2d(1, 50, kernel_size=(1, 200), stride=(1, 1))
  )
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc1): Linear(in_features=150, out_features=4, bias=True)
)

강남구 리뷰 데이터를 통한 테스트


In [ ]:
test_result = []

In [ ]:
result = pd.DataFrame()

In [ ]:
response = test.mbti

In [ ]:
intent_list

['EN', 'ES', 'IN', 'IS']

In [ ]:
test_result = []

In [ ]:
#테스트할때
a=0
for idx in list(test.text):
    if idx == '0':
      test_result.append('no data')
      print(a)
      a += 1
      continue
    line = mecabsplit(tagger,idx, False)
  
    cont = []
    for it in line:
        if it in content_vocab:
            cont.append(content_vocab[it])  # cont에는 입력문에 나타난 단어들의 index 저장됨
        else:
            cont.append(content_vocab['unk'])
    
    cont = Variable(torch.LongTensor(cont)).view(1,-1)
    pred = model(cont)
    #(1, C)
    v,i = torch.max(pred,1) # i: 가장 확률 높은 클래스의 인덱스
    
    print('input : ', line)
    print('intent : ', intent_list[int(i)])
    test_result.append(intent_list[int(i)])
    print(int(i))
    # print('answer : ', response[a],'\n')
    print(a)
    a += 1

input :  ['목록', '화살', '의자']
intent :  IS
3
0
input :  ['목수', '화신', '이', '랴']
intent :  IS
3
1
input :  ['목', '에', '화씨', '이론']
intent :  EN
0
2
input :  ['목자', '환희', '이름']
intent :  IS
3
3
input :  ['낮', '게', '순종', '초반']
intent :  IS
3
4
input :  ['낳', '다', '순하', 'ᆫ', '초안']
intent :  IN
2
5
input :  ['내선', '순회', '초월']
intent :  IN
2
6
input :  ['내성', '술부', '초점']
intent :  IN
2
7
input :  ['마빈', '연합', '폭죽']
intent :  ES
1
8
input :  ['마샬', '열소', '폭탄']
intent :  ES
1
9
input :  ['마오', '염소', '폭풍']
intent :  ES
1
10
input :  ['마을', '엽서', '폭행']
intent :  ES
1
11
input :  ['무게', '오턴', '헤겔']
intent :  EN
0
12
input :  ['무기', '오', '흐', '헤더']
intent :  EN
0
13
input :  ['무나', '옥상', '헤라']
intent :  EN
0
14
input :  ['무례', '옥시', '헤헤']
intent :  EN
0
15


In [ ]:
test_result

['IS',
 'IS',
 'EN',
 'IS',
 'IS',
 'IN',
 'IN',
 'IN',
 'ES',
 'ES',
 'ES',
 'ES',
 'EN',
 'EN',
 'EN',
 'EN']

In [ ]:
result = pd.DataFrame()

In [ ]:
result["원본"] = test.mbti

In [ ]:
result["예측"] = test_result

In [ ]:

result

,원본,예측
0,IS,IS
1,IS,IS
2,IS,EN
3,IS,IS
4,IN,IS
5,IN,IN
6,IN,IN
7,IN,IN
8,ES,ES
9,ES,ES


In [ ]:
from sklearn.metrics import f1_score

In [ ]:
print(f1_score(result["원본"], result["예측"],average='micro'))
print(f1_score(result["원본"], result["예측"],average='weighted'))
print(f1_score(result["원본"], result["예측"],average='macro'))

0.875
0.8740079365079365
0.8740079365079365


In [ ]:
from sklearn.metrics import classification_report

In [ ]:
report = classification_report(result["원본"], result["예측"])

In [ ]:
print(report)

              precision    recall  f1-score   support

          EN       0.80      1.00      0.89         4
          ES       1.00      1.00      1.00         4
          IN       1.00      0.75      0.86         4
          IS       0.75      0.75      0.75         4

    accuracy                           0.88        16
   macro avg       0.89      0.88      0.87        16
weighted avg       0.89      0.88      0.87        16



In [ ]:
test=pd.read_csv('/content/gdrive/MyDrive/빅프/test_gangnam.csv',encoding='utf-8', on_bad_lines='skip')

In [ ]:
#리뷰돌릴때
a=0
for text in list(test.columns):
  for idx in list(test[text]):
      if idx == '0':
        test_result.append('no data')
        print(a)
        a += 1
        continue
      line = mecabsplit(tagger,idx, False)
    
      cont = []
      for it in line:
          if it in content_vocab:
              cont.append(content_vocab[it])  # cont에는 입력문에 나타난 단어들의 index 저장됨
          else:
              cont.append(content_vocab['unk'])
      
      cont = Variable(torch.LongTensor(cont)).view(1,-1)
      pred = cnn(cont)
      #(1, C)
      v,i = torch.max(pred,1) # i: 가장 확률 높은 클래스의 인덱스
      
      print('input : ', line)
      print('intent : ', intent_list[int(i)])
      test_result.append(intent_list[int(i)])
      print(int(i))
      # print('answer : ', response[a],'\n')
      print(a)
      a += 1
  result[text] = test_result
  test_result = []

input :  ['목록', '화살', '의자']
intent :  IS
3
0
input :  ['목수', '화신', '이', '랴']
intent :  IS
3
1
input :  ['목', '에', '화씨', '이론']
intent :  IS
3
2
input :  ['목자', '환희', '이름']
intent :  ES
1
3
input :  ['낮', '게', '순종', '초반']
intent :  IN
2
4
input :  ['낳', '다', '순하', 'ᆫ', '초안']
intent :  IN
2
5
input :  ['내선', '순회', '초월']
intent :  IN
2
6
input :  ['내성', '술부', '초점']
intent :  IN
2
7
input :  ['마빈', '연합', '폭죽']
intent :  ES
1
8
input :  ['마샬', '열소', '폭탄']
intent :  ES
1
9
input :  ['마오', '염소', '폭풍']
intent :  ES
1
10
input :  ['마을', '엽서', '폭행']
intent :  ES
1
11
input :  ['무게', '오턴', '헤겔']
intent :  IS
3
12
input :  ['무기', '오', '흐', '헤더']
intent :  EN
0
13
input :  ['무나', '옥상', '헤라']
intent :  EN
0
14
input :  ['무례', '옥시', '헤헤']
intent :  EN
0
15
input :  ['IS']
intent :  ES
1
16
input :  ['IS']
intent :  ES
1
17
input :  ['IS']
intent :  ES
1
18
input :  ['IS']
intent :  ES
1
19
input :  ['IN']
intent :  ES
1
20
input :  ['IN']
intent :  ES
1
21
input :  ['IN']
intent :  ES
1
22
input :  ['

In [ ]:
result

,text,mbti
0,IS,ES
1,IS,ES
2,IS,ES
3,ES,ES
4,IN,ES
5,IN,ES
6,IN,ES
7,IN,ES
8,ES,ES
9,ES,ES


In [ ]:
torch.save(cnn,'/content/gdrive/My Drive/빅프/cnn_model_v7.pt')

In [ ]:
#기타

In [ ]:
#리뷰데이터 불러오기
test=pd.read_csv('/content/gdrive/MyDrive/빅프/review/total_total.csv',encoding='utf-8', on_bad_lines='skip')

In [ ]:
test.head(1)

,search_key,name_x,category,rating,address,tel,year_10,year_20,year_30,year_40,...,review_blog1,review_blog2,review_blog3,review_blog4,review_blog5,review_blog6,review_blog7,review_blog8,review_blog9,review_blog10
0,한식,떡도리탕 강남본점,닭볶음탕,4.47,서울 강남구 테헤란로1길 28-9 1층,0507-1352-2940,29.55,75.06,49.99,28.73,...,강남 맛집 풍자가 선택한 또간집 떡도리탕 내돈내산 후기|회사 점심시간에 유튜버이자 ...,11.28~12.4 주간 일기|벌써 주간 일기가 마지막 주간이구나. 이렇게 오래 쓰...,강남역 맛집 떡도리탕 주말 웨이팅 성공|강남맛집 떡도리탕입니다. 한 번의 실패를 맛...,[서울] 떡도리탕 강남본점 - 강남역 신논현역 닭볶음탕 + 떡볶이|언니야가 한번 가...,강남 맛집 또간집 떡도리탕 웨이팅|안녕하세요 재교입니다. 강남 맛집 또간집 떡도리탕...,[내돈내산] 강남 떡도리탕 풍자 또간집 맛집 방문 웨이팅 후기|오랜만에 강남에서 계...,떡도리탕 강남본점 또간집 웨이팅후기|안녕하세요 코코입니다 평소 유튜브로 또간집을 즐...,강남역 맛집 떡도리탕 평일 웨이팅 꿀팁 / 풍자 또간집|오늘은 요즘 강남역의 핫한 ...,강남 떡도리탕 평일 웨이팅 예약방법 /풍자 또간집 맛집|최근에 관심있게 보는 유튜브...,강남 떡도리탕 오픈런 성공후기 :)|어제는 친구들을 만났지. 출산전에 꼭 한번 만나...


In [ ]:
test.columns

Index(['search_key', 'name_x', 'category', 'rating', 'address', 'tel',
       'year_10', 'year_20', 'year_30', 'year_40', 'year_50', 'year_60',
       'male', 'female', 'detail_url', 'gu', 'place_num', 'url', 'name_y',
       'main_jpg', 'review_visitor_count', 'review_blog_count',
       'review_summary1', 'review_summary1_cnt', 'review_summary2',
       'review_summary2_cnt', 'review_summary3', 'review_summary3_cnt',
       'review_summary4', 'review_summary4_cnt', 'review_summary5',
       'review_summary5_cnt', 'keyword_detail', 'review1', 'review2',
       'review3', 'review4', 'review5', 'review6', 'review7', 'review8',
       'review9', 'review10', 'review_blog1', 'review_blog2', 'review_blog3',
       'review_blog4', 'review_blog5', 'review_blog6', 'review_blog7',
       'review_blog8', 'review_blog9', 'review_blog10'],
      dtype='object')

In [ ]:
test = test.drop(['search_key', 'name_x', 'category', 'rating', 'address', 'tel',
       'year_10', 'year_20', 'year_30', 'year_40', 'year_50', 'year_60',
       'male', 'female', 'detail_url', 'gu', 'place_num', 'url', 'name_y',
       'main_jpg', 'review_visitor_count', 'review_blog_count','review_summary1', 'review_summary1_cnt', 'review_summary2',
       'review_summary2_cnt', 'review_summary3', 'review_summary3_cnt',
       'review_summary4', 'review_summary4_cnt', 'review_summary5',
       'review_summary5_cnt', 'keyword_detail'], axis=1)

In [ ]:
#결측치 채우기
test = test.fillna('0')

In [ ]:
test_result = []

In [ ]:
result = pd.DataFrame()

In [ ]:
test.columns

Index(['review1', 'review2', 'review3', 'review4', 'review5', 'review6',
       'review7', 'review8', 'review9', 'review10', 'review_blog1',
       'review_blog2', 'review_blog3', 'review_blog4', 'review_blog5',
       'review_blog6', 'review_blog7', 'review_blog8', 'review_blog9',
       'review_blog10'],
      dtype='object')

In [ ]:
a=0
for text in list(test.columns):
  for idx in list(test[text]):
      if idx == '0' or idx == ' ' or idx == '\n' or idx == '||':
        test_result.append('no data')
        print(a)
        a += 1
        continue
      line = mecabsplit(tagger,idx, False)
    
      cont = []
      for it in line:
          if it in content_vocab:
              cont.append(content_vocab[it])  # cont에는 입력문에 나타난 단어들의 index 저장됨
          else:
              cont.append(content_vocab['unk'])
      
      cont = Variable(torch.LongTensor(cont)).view(1,-1)
      pred = cnn(cont)
      #(1, C)
      v,i = torch.max(pred,1) # i: 가장 확률 높은 클래스의 인덱스
      
      print('input : ', line)
      print('intent : ', intent_list[int(i)])
      test_result.append(intent_list[int(i)])
      print(int(i))
      # print('answer : ', response[a],'\n')
      print(a)
      a += 1
  result[text] = test_result
  test_result = []

Streaming output truncated to the last 5000 lines.
input :  ['[', '이', '촌역', '/', '국립', '중앙', '박물', '관', '맛', '집', ']', '"', '칸', '티나', '"', '-', '맛있', '는', '지중', '하', '아', '가', '아', '정식', '을', '맛보', 'ᆯ', '.', '..', '|', 'https', ':', '/', '/', 'www', '.', 'instagram', '.', 'com', '/', 'kantina', '_', 'seoul', '/', '위치', '및', '영업', '시간', '위치', '는', '이', '촌역', '바로', '앞', ',', '파크타워', '아파트', '상가', '1', '층', '에', '위치', '하', '아', '있', '다', '그래서', '주차', '도', '아파트', '주차', '장', '에다가', '하', '면', '되', 'ᆫ다', ':', ')', '영업', '시간', '매주', '월', '요일', '휴무', '화', '-', '토', '요일', ':', '11', ':', '30', '-', '22', ':', '00', '.', '..', '|', 'https', ':', '/', '/', 'blog', '.', 'naver', '.', 'com', '/', 'moonsu', '0628', '/', '222850234976']
intent :  IN
2
252687
input :  ['녹사평', '프렌치', '퀴진', '셰', '누', '프라이빗', '키친', '|', '2022', '상', '반기', '팀', '디너', '로', '다녀오', 'ᆫ', '셰', '누', '프라이빗', '키친', '!', '정통', '프렌치', '느낌', '보다', 'ᆫ', '셰프', '님', '의', '스타일', '이', '묻어나', '는', '트렌디', '한', '프렌치', '퀴진', '이', 'ᆫ데', ',', 

In [ ]:
result

,review1,review2,review3,review4,review5,review6,review7,review8,review9,review10,review_blog1,review_blog2,review_blog3,review_blog4,review_blog5,review_blog6,review_blog7,review_blog8,review_blog9,review_blog10
0,EN,IN,ES,EN,IN,IN,IN,IN,IN,IN,IN,IS,IN,IN,IN,IN,IN,IN,EN,ES
1,EN,IN,IS,IN,ES,IN,IN,EN,EN,EN,IN,ES,IN,IN,IN,ES,IN,IN,IN,IN
2,IN,EN,IN,IN,IN,IN,IN,IN,IN,IN,IN,IN,IN,IN,ES,IN,IN,IN,EN,IN
3,IN,IN,IN,EN,ES,IN,IN,IN,IN,IN,IN,IN,ES,IN,IN,IN,IN,EN,IN,ES
4,IN,EN,IN,IN,IN,IN,IN,IN,EN,EN,EN,EN,IN,ES,IN,IN,IN,IN,IN,IN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12719,IS,IN,IS,IS,IS,IS,no data,no data,no data,no data,ES,IN,IN,IN,no data,no data,no data,no data,no data,no data
12720,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data
12721,ES,no data,no data,no data,no data,no data,no data,no data,no data,no data,IN,IN,IN,no data,no data,no data,no data,no data,no data,no data
12722,no data,no data,no data,no data,no data,no data,no data,no data,no data,no data,IS,IN,IN,ES,IN,IN,IN,EN,EN,no data


In [ ]:
result.to_csv('/content/gdrive/MyDrive/빅프/review/total_mbti_v2.csv')